# 🔍 AI-Powered Data Scientist Job Search Engine  
*Personalized Job Matching with Gemini + Semantic Search*

---

## 👋 Introduction

As I near graduation from the **Master of Management Analytics program at Smith School of Business, Queen’s University**, I find myself navigating the increasingly competitive world of data science careers. This program is widely regarded as Canada’s equivalent to a master’s degree in data science, and it has equipped me with deep analytical, AI, and machine learning skills.

But one real-world challenge still remains: **finding the right job**. Scanning hundreds of listings, manually evaluating job descriptions, comparing roles with my unique profile—it’s time-consuming, inconsistent, and inefficient.

This inspired me to tackle a **problem that’s deeply personal and timely**:  
> 📌 **Can we use GenAI to intelligently search, evaluate, and recommend jobs that align with our exact strengths and goals—just like a personalized job coach?**

---

## 🛠️ What This Project Does

This Capstone project presents an **AI-powered job recommendation engine** that leverages the capabilities of the Google Gemini API. It scrapes real-time job listings, embeds job descriptions using LLM-generated vectors, matches them with the candidate’s profile, and generates structured recommendations with **natural-language reasoning** on why each job is a fit.

This isn’t just a prototype—it’s my actual job search assistant, built with the tools and techniques I’ve learned from the **5-Day GenAI Intensive Course by Google & Kaggle**.

---

## ✅ Capstone Guidelines Fulfilled

### 🎯 Real-World Use Case
- Directly solves a meaningful problem for data/AI graduates and professionals
- Personalized, practical, and scalable

### 🧠 GenAI Capabilities Demonstrated
- **Embeddings** – using `text-embedding-004` to encode job descriptions and match them to user profiles
- **Structured Output (JSON Mode)** – generating concise, schema-based job-to-candidate match justifications
- **Semantic Similarity & Ranking** – vector-based scoring of job relevance using Gemini + ChromaDB

### 🖥️ Additional Features
- Prioritization by **recency** and **location relevance**
- Clean, dark-themed **HTML output** for professional display
- Rate-limit friendly execution with Gemini’s usage patterns in mind

---

> This is more than a school project—it's a working, real-life solution built at the intersection of career needs and GenAI innovation.


In [22]:
!pip install -qU google-genai chromadb requests beautifulsoup4 > /dev/null 2>&1

## 🔐 Set Up Gemini API via Kaggle Secrets

In [23]:

from kaggle_secrets import UserSecretsClient
import os
from google import genai

secrets = UserSecretsClient()
GOOGLE_API_KEY = secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)


## 🌐 Scrape LinkedIn for Data Scientist Jobs

In [24]:

import requests
from bs4 import BeautifulSoup

def scrape_linkedin_jobs(query, location="Canada", pages=3):
    jobs = []
    for page in range(pages):
        url = f"https://ca.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={query}&location={location}&start={page*25}"
        headers = {"User-Agent": "Mozilla/5.0"}
        soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")
        for job in soup.select(".base-card"):
            if job.select_one(".base-search-card__title"):
                jobs.append({
                    "title": job.select_one(".base-search-card__title").text.strip(),
                    "company": job.select_one(".base-search-card__subtitle").text.strip(),
                    "location": job.select_one(".job-search-card__location").text.strip(),
                    "link": job.select_one("a")['href'],
                    "posted": job.select_one("time")['datetime'] if job.select_one("time") else "Unknown",
                    "salary": job.select_one(".salary-snippet").text.strip() if job.select_one(".salary-snippet") else "Not listed"
                })
    return jobs

jobs = scrape_linkedin_jobs("Data Scientist")
len(jobs)


30

## 🧠 Embed Job Descriptions for Semantic Matching

In [25]:
import chromadb
from chromadb import Documents, EmbeddingFunction
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        pass  # optional: initialize anything here later

    def __call__(self, input: Documents):
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(task_type="semantic_similarity")
        )
        return [e.values for e in response.embeddings]


db = chromadb.Client().get_or_create_collection("jobs", embedding_function=GeminiEmbeddingFunction())
docs = [f"{job['title']} at {job['company']} in {job['location']} posted on {job['posted']}" for job in jobs]
db.add(documents=docs, ids=[str(i) for i in range(len(jobs))])


## 👤 Define Candidate Profile

In [26]:
user_profile = '''
Data Scientist skilled in Python, machine learning, AI, NLP, TensorFlow, PyTorch, SQL, AWS, Azure, Google Cloud. Recently graduated.
'''

## 🚦 Prioritize and Recommend Jobs

In [27]:
def prioritize(job):
    location_priority = ["North York", "Toronto", "Remote", "Ontario", "Canada"]
    location_score = next((i+1 for i, loc in enumerate(location_priority) if loc.lower() in job["location"].lower()), 6)
    
    from datetime import datetime
    try:
        days_ago = (datetime.now() - datetime.fromisoformat(job["posted"])).days if "T" in job["posted"] else 999
    except Exception:
        days_ago = 999

    time_score = 1 if days_ago <= 1 else 2 if days_ago <= 7 else 3
    return location_score, time_score

# ✅ Safely limit to available jobs
num_results = min(20, len(jobs))  # Adjust based on what's embedded

query_result = db.query(query_texts=[user_profile], n_results=num_results)

# ✅ Filter out invalid indices
top_jobs = sorted(
    [jobs[int(idx)] for idx in query_result["ids"][0] if int(idx) < len(jobs)],
    key=prioritize
)


## ✨ Generate Match Reasons (Top 5 Only to Respect API Quota)

In [28]:

import time

structured_recommendations = []
for i, job in enumerate(top_jobs[:5]):
    prompt = f'''
    Candidate recently graduated, skilled in AI/ML/cloud. Job: {job}
    Provide one concise sentence on match suitability.
    '''
    response = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
    structured_recommendations.append({
        **job,
        "relevance_reason": response.text.strip()
    })
    time.sleep(5)  # Avoid rate limit


## 🌟 Display Results in Styled HTML

In [29]:

import pandas as pd
from IPython.display import HTML

df = pd.DataFrame(structured_recommendations)
df['link'] = df['link'].apply(lambda x: f'<a href="{x}" target="_blank">Job Link</a>')
df = df[['title', 'company', 'location', 'posted', 'salary', 'link', 'relevance_reason']].rename(columns=str.title).sort_values('Posted', ascending=False)

styled_html = f'''
<style>
    body {{background:#121212;color:white;}}
    table {{width:100%;border-collapse:collapse;}}
    th,td {{padding:10px;border:1px solid #444;}}
    th {{background:#333;}}
    a {{color:#4ea1ff;}}
</style>
{df.to_html(escape=False, index=False)}
'''
HTML(styled_html)


Title,Company,Location,Posted,Salary,Link,Relevance_Reason
Data Scientist,Deloitte,"Toronto, Ontario, Canada",2025-04-20,Not listed,Job Link,"The candidate's AI/ML/cloud skills align well with a Data Scientist role, making them a potentially suitable fit."
Data Scientist(GenAI),Tiger Analytics,"Toronto, Ontario, Canada",2025-04-10,Not listed,Job Link,This candidate's skills in AI/ML/cloud align well with the Data Scientist (GenAI) role at Tiger Analytics.
Data Scientist,LatentView Analytics,"Toronto, Ontario, Canada",2025-04-04,Not listed,Job Link,"This candidate is a potentially good fit given their AI/ML/cloud skills align with Data Scientist roles, but further investigation is needed to assess experience level against job requirements."
Data Scientist,Equifax,"Toronto, Ontario, Canada",2025-04-01,Not listed,Job Link,This candidate appears to be a potentially good fit due to their AI/ML/cloud skills aligning with the Data Scientist role at Equifax.
Data Scientist,Ample Insight,"Toronto, Ontario, Canada",2025-03-25,Not listed,Job Link,This candidate's skills in AI/ML/cloud make them a potentially good fit for the Data Scientist role at Ample Insight.
